550. Game Play Analysis IV
Table: Activity

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| player_id    | int     |
| device_id    | int     |
| event_date   | date    |
| games_played | int     |
+--------------+---------+
(player_id, event_date) is the primary key of this table.
This table shows the activity of players of some games.
Each row is a record of a player who logged in and played a number of games (possibly 0) before logging out on someday using some device.
 

Write an SQL query to report the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.

In [ ]:
# Write your MySQL query statement below
SELECT
    ROUND(count(distinct player_id) / (SELECT count(distinct player_id) FROM Activity), 2) as fraction
FROM Activity
WHERE (player_id, event_date) in (SELECT player_id, DATE(min(event_date)+1) FROM Activity GROUP BY player_id)

262. Trips and Users
Table: Trips

+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| id          | int      |
| client_id   | int      |
| driver_id   | int      |
| city_id     | int      |
| status      | enum     |
| request_at  | date     |     
+-------------+----------+
id is the primary key for this table.
The table holds all taxi trips. Each trip has a unique id, while client_id and driver_id are foreign keys to the users_id at the Users table.
Status is an ENUM type of ('completed', 'cancelled_by_driver', 'cancelled_by_client').
 

Table: Users

+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| users_id    | int      |
| banned      | enum     |
| role        | enum     |
+-------------+----------+
users_id is the primary key for this table.
The table holds all users. Each user has a unique users_id, and role is an ENUM type of ('client', 'driver', 'partner').
banned is an ENUM type of ('Yes', 'No').
 

The cancellation rate is computed by dividing the number of canceled (by client or driver) requests with unbanned users by the total number of requests with unbanned users on that day.

Write a SQL query to find the cancellation rate of requests with unbanned users (both client and driver must not be banned) each day between "2013-10-01" and "2013-10-03". Round Cancellation Rate to two decimal points.

Return the result table in any order.

In [ ]:
SELECT
    request_at as Day, ROUND(SUM(IF(status = 'cancelled_by_driver' OR status = 'cancelled_by_client', 1,0)) / count(*), 2) as 'Cancellation Rate'
FROM Trips
WHERE driver_id in (SELECT users_id FROM Users WHERE banned = 'No') and client_id in (SELECT users_id FROM Users WHERE banned = 'No') and request_at BETWEEN "2013-10-01" AND "2013-10-03"
GROUP BY request_at
ORDER BY Day

In [ ]:
# Write your MySQL query statement below
with stats as (
  select 
    Request_at, 
    T.Status <> 'completed' as IsCancelled
  from Trips T 
  join Users C on (Client_Id = C.Users_Id and C.Banned = 'No') 
  join Users D on (Driver_Id = D.Users_Id and D.Banned = 'No') 
  where
    Request_at between '2013-10-01' and '2013-10-03'
)
select 
  Request_at as Day,
  Round(
    cast(sum(IsCancelled) as real) / cast(count(*) as real),
    2
  ) as 'Cancellation Rate'
from stats
group by Request_at